In [1]:
# "작은 별", "봄나들이", "나비야 나비야" 악보를 ABC 표기로 표현
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2"
spring_picnic="tinynotation: 4/8 g8 e8 g8 e8 g8 a8 g4 e8 g8 e8 c8 d8 e8 c4 g8 e8 g8 e8 g8 a8 g4 b8 a8 g8 e8 d8 e8 c4"
butterfly="tinynotation: 2/4 g8 e8 e4 f8 d8 d4 c8 d8 e8 f8 g8 g8 g4 g8 e8 e8 e8 f8 d8 d4 c8 e8 g8 g8 e8 e8 e4 d8 d8 d8 d8 d8 e8 f4 e8 e8 e8 e8 e8 f8 g4 g8 e8 e4 f8 d8 d4 c8 e8 g8 g8 e8 e8 e4"

In [6]:
# 계이름과 숫자를 상호 변환하는 표(딕셔너리 자료구조를 사용함) - 규칙
import numpy as np
note2num={'c':1,'d':2,'e':3,'f':4,'g':5,'a':6,'b':7}
num2note={1:'c',2:'d',3:'e',4:'f',5:'g',6:'a',7:'b'}
# ABC를 시계열로 변환
def abcToTimesSeries(s):
    note =  s.split(' ')[2:]    # 데이터를 3번째 행부터 추출 
    return [ [note2num[i[0]], int(i[1])]  for i in note ]

# 시계열 데이터를 ABC 표기로 변환(RollBack - 원상태로 돌려놓음(원상복귀))
def timesToAbcSeries(data):
    s = 'tinynotation: 4/4'
    for i in data:
        s += ' '+ num2note[i[0]]+str(i[1])
    return s

# 원핫 기준코드
onehot=[[1,2],[2,2],[3,2],[4,2],[5,2],[6,2],[7,2],[1,4],[2,4],[3,4],[4,4],[5,4],[6,4],[7,4],[1,8],[2,8],[3,8],[4,8],[5,8],[6,8],[7,8]]
def onehotMake(ydata):
    t=[]
    for i in range(len(ydata)):
        a = np.zeros(len(onehot))
        a[onehot.index(list(ydata[i]))] = 1.0
        t.append(a)
    return np.array(t)

# LSTM 형태로 데이터를 만듦
# 시계열 데이터를 윈도우 단위로 자르는 함수
def seq2dataset(seq,window,horizon):
    X=[]; Y=[]
    for i in range(len(seq)-(window+horizon)+1):
        x=seq[i:(i+window)]
        y=(seq[i+window+horizon-1])
        X.append(x); Y.append(y)
    return np.array(X), np.array(Y)

w=7 # 윈도우 크기
h=1 # 수평선 계수

seq1 = abcToTimesSeries(little_star)
seq2 = abcToTimesSeries(spring_picnic)
seq3 = abcToTimesSeries(butterfly)
seq = seq1+seq2+seq3

X,Y= seq2dataset(seq,w,h)

In [7]:
# LSTM 신경망 구성하고.. 데이터 학습해서.. 첫소절을 윈도우 사이즈보다 크게 해서 새롭게 분류된 음표들을 출력
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import tensorflow as tf
split=int(len(X)*1.0) # 100%를 훈련집합으로 사용
x_train=X[0:split]; y_train=Y[0:split]
y_train=onehotMake(y_train)
# LSTM 모델 설계와 학습
model=Sequential()
model.add(LSTM(units=128,activation='relu',input_shape=x_train[0].shape))
model.add(Dense(y_train.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=300,batch_size=1,verbose=2)

Epoch 1/300
117/117 - 1s - loss: 2.6174 - accuracy: 0.1709 - 803ms/epoch - 7ms/step
Epoch 2/300
117/117 - 0s - loss: 2.3617 - accuracy: 0.2821 - 176ms/epoch - 2ms/step
Epoch 3/300
117/117 - 0s - loss: 2.2494 - accuracy: 0.2991 - 179ms/epoch - 2ms/step
Epoch 4/300
117/117 - 0s - loss: 2.2867 - accuracy: 0.2991 - 174ms/epoch - 1ms/step
Epoch 5/300
117/117 - 0s - loss: 2.1437 - accuracy: 0.2906 - 172ms/epoch - 1ms/step
Epoch 6/300
117/117 - 0s - loss: 2.0792 - accuracy: 0.3333 - 173ms/epoch - 1ms/step
Epoch 7/300
117/117 - 0s - loss: 2.0626 - accuracy: 0.3248 - 199ms/epoch - 2ms/step
Epoch 8/300
117/117 - 0s - loss: 1.9816 - accuracy: 0.3333 - 191ms/epoch - 2ms/step
Epoch 9/300
117/117 - 0s - loss: 1.9487 - accuracy: 0.3675 - 207ms/epoch - 2ms/step
Epoch 10/300
117/117 - 0s - loss: 1.8668 - accuracy: 0.3504 - 179ms/epoch - 2ms/step
Epoch 11/300
117/117 - 0s - loss: 1.8337 - accuracy: 0.3590 - 175ms/epoch - 1ms/step
Epoch 12/300
117/117 - 0s - loss: 1.8224 - accuracy: 0.3675 - 198ms/epoch 

In [10]:
# 학습된 인공지능 모델을이용해서 첫음절을 투입하면 학습한 알고리즘에 의해서 (기존에 잇는 음표중에 분류모델)
def makeNewSong(start,duration):
    for i in range(duration):
        new_song = []
        a = np.expand_dims( start[i:],axis=0)    
        p= model.predict(a)
        new_song.append( onehot[np.argmax(p)])
        start = np.append(start,new_song,axis=0)        
    return timesToAbcSeries(start)

In [18]:
first_song = [
    [1,2],[3,2],[3,2],[4,2],[1,2],[1,2],[1,2]
]
np.array(first_song).shape
maked_new_song = makeNewSong(first_song,30)

In [19]:
import music21
music21.converter.parse(maked_new_song).show('mid')